# Deep Convolutional Generative Adversarial Network

In [1]:
from __future__ import print_function
import argparse
import os
import random
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
%matplotlib inline
from IPython.display import HTML

In [2]:
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.utils.data

In [3]:
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torchvision.utils as vutils

In [4]:
# set random seed for reproducibility
manual_seed = 1
torch.manual_seed(manual_seed)
random.seed(manual_seed)

## Set Arguments

In [5]:
# root directory for dataset
data_root = 'data/celeba'

# number of workers for data loader
workers = 2

# batch size during training
batch_size = 128

# spatial size of training images
# all images will be resized to this size using a transformer
image_size = 64

# number of channels in the training images
n_channels = 3

# size of Z latent vector (i.e. size of generator input)
n_zlvector = 100

# size of feature maps in generator
n_fgenerator = 64

# size of feature maps in discriminator
n_fdiscriminator = 64

# number of training epochs
n_epochs = 5

# learning rate for optimizers
lr = 0.0002

# beta1 hyper-param for Adam optimizers
beta1 = 0.5
beta2 = 0.999

# number of GPUs availabele
n_gpu = 1

## Configure Data Loader

In [6]:
# we can use an image folder dataset the way we have it setup
data_loader = torch.utils.data.DataLoader(
    datasets.ImageFolder(root = data_root,
                         transform = transforms.Compose([
                             transforms.Resize(image_size),
                             transforms.CenterCrop(image_size),
                             transforms.ToTensor(),
                             transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                         ])),
                         batch_size = batch_size,
                         shuffle = True,
                         num_workers = workers
)

In [7]:
# decide which device we want to run on
device = torch.device('cuda:0' if (torch.cuda.is_available() and n_gpu > 0) else 'cpu')

In [8]:
real_batch = next(iter(data_loader))
plt.figure(figsize=(10,10))
plt.axis('off')
plt.title('Training Images (Celeb-A Faces Dataset)')
plt.imshow(np.transpose(vutils.make_grid(real_batch[0].to(device)[:64], padding=2, normalize=True).cpu(), (1, 2, 0)))
plt.show()

## Build DCGAN Architecture

<img src='images/dcgan-architecture.png' width='75%'/>

In [9]:
# custome weights initialization called on the generator and discriminator
def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find('conv') != -1:
        torch.nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm2d') != -1:
        torch.nn.init.normal_(m.weight.data, 1.0, 0.02)
        torch.nn.init.constant_(m.bias.data, 0.0)

## Generator

In [10]:
class Generator(nn.Module):
    
    def __init__(self, n_gpu):
        super(Generator, self).__init__()
        
        self.n_gpu = n_gpu
        self.conv_blocks = nn.Sequential(
            nn.ConvTranspose2d(n_zlvector, n_fgenerator * 8, kernel_size=4, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(n_fgenerator * 8),
            nn.ReLU(True),
            
            nn.ConvTranspose2d(n_fgenerator * 8, n_fgenerator * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(n_fgenerator * 4),
            nn.ReLU(True),
            
            nn.ConvTranspose2d(n_fgenerator * 4, n_fgenerator * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(n_fgenerator * 2),
            nn.ReLU(True),
            
            nn.ConvTranspose2d(n_fgenerator * 2, n_fgenerator, 4, 2, 1, bias=False),
            nn.BatchNorm2d(n_fgenerator),
            nn.ReLU(True),
            nn.ConvTranspose2d(n_fgenerator, n_channels, 4, 2, 1, bias=False),
            nn.Tanh(),
        )
    
    def forward(self, z):
        img = self.conv_blocks(z)
        return img

## Discriminator

In [11]:
class Discriminator(nn.Module):
    
    def __init__(self, n_gpu):
        super(Discriminator, self).__init__()
        self.n_gpu = n_gpu
        self.model = nn.Sequential(
            nn.Conv2d(n_channels, n_fdiscriminator, kernel_size=4, stride=2, padding=1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Conv2d(n_fdiscriminator, n_fdiscriminator*2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(n_fdiscriminator * 2),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Conv2d(n_fdiscriminator*2, n_fdiscriminator*4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(n_fdiscriminator*4),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Conv2d(n_fdiscriminator*4, n_fdiscriminator*8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(n_fdiscriminator*8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(n_fdiscriminator*8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid(),
        )
    
    def forward(self, img):
        validity = self.model(img)
        return validity

#### Initialize the generator and discriminator

In [12]:
generator = Generator(n_gpu).to(device)
discriminator = Discriminator(n_gpu).to(device)

# handle multi-gpu if desired
if (device.type == 'cuda') and (n_gpu > 1):
    generator = nn.DataParallel(generator, list(range(n_gpu)))
    discriminator = nn.DataParallel(discriminator, list(range(n_gpu)))

## Loss Function

In [13]:
adversarial_loss = nn.BCELoss()
if (device.type == 'cuda'): adversarial_loss.cuda()

## Optimizer

In [14]:
optimizer_G = torch.optim.Adam(generator.parameters(), lr=lr, betas=(beta1, beta2))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=lr, betas=(beta1, beta2))

## Training The [Network](https://arxiv.org/pdf/1511.06434)

#### Initialize weights

In [15]:
generator.apply(weights_init_normal)

Generator(
  (conv_blocks): Sequential(
    (0): ConvTranspose2d(100, 512, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU(inplace=True)
    (12): ConvTranspose2d(64, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (13): Tanh()

In [16]:
discriminator.apply(weights_init_normal)

Discriminator(
  (model): Sequential(
    (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): LeakyReLU(negative_slope=0.2, inplace=True)
    (8): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (9): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): LeakyReLU(negative_slope=0.2, inplace=True)
    (11): Conv2d(512, 1, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (12): Sigmoid()
  )
)

In [17]:
img_list = []
g_loss_to_display = []
d_loss_to_display = []
iters = 0

for epoch in range(n_epochs):
    for i, (imgs, _) in enumerate(data_loader):
        print(imgs.shape)
        break
    break

torch.Size([128, 3, 64, 64])


In [ ]:
plt.figure(figsize=(10,5))
plt.title("Generator and Discriminator Loss During Training")
plt.plot(g_loss_to_display, label="Generator Loss")
plt.plot(d_loss_to_display, label="Discriminator Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
# plt.savefig('./images/dcgan/final_loss.png')
plt.show()

## Final GAN Images

In [ ]:
%%capture
fig = plt.figure(figsize=(8,8))
plt.axis("off")
ims = [[plt.imshow(np.transpose(i,(1,2,0)), animated=True)] for i in img_list]
ani = animation.ArtistAnimation(fig, ims, interval=1000, repeat_delay=1000, blit=True)

HTML(ani.to_jshtml())

In [ ]:
# grab a batch of real images from the data loader
real_batch = next(iter(data_loader))

# plot the real images
plt.figure(figsize=(15,15))
plt.subplot(1,2,1)
plt.axis("off")
plt.title("Real Images")
plt.imshow(np.transpose(vutils.make_grid(real_batch[0].to(device)[:64], padding=5, normalize=True).cpu(),(1,2,0)))

# plot the fake images from the last epoch
plt.subplot(1,2,2)
plt.axis("off")
plt.title("Fake Images")
plt.imshow(np.transpose(img_list[-1],(1,2,0)))
plt.show()

---